# Transportligning

I innføringen kalte vi ligningen

$$
u_x = u_y
$$

for en transportligning. La oss nå omdøpe variablene for $x$ og $t$. En mer generell versjon av samme ligning er

$$
u_t + a u_x = 0
$$

Det er ligningen som vi skal studere her. Senere vil vi se på den enda mer generell ligning

$$
u_t + a(x,t,u)u_x = g(x,t),
$$

og den nær beslektet *bevaringslov*

$$
u_t + f(x,t,u)_x = g(x,t)
$$

## 1. En analytisk løsning

**Oppgave** sjekk med derivasjon at $u(x,t) = g(x-at)$ løser ligningen for hvilken som helst funksjon.

### a) Med initalbetingelser

Anta at vi har en initialbetingelse av formen $u(x,0) = f(x)$.

Sammenlign det med løsningen $u(x,t) = g(x-at)$; vi setter inn $t=0$ og får $u(x,0)=g(x)$. Dvs, vi må ta samme funksjon $g(\cdot)$ som den initialbetingelse. Løsningen er altså

$$
u(x,t) = f(x-at)
$$

Hva betyr det? Vi ser at løsningen flytter på seg, med hastighet (inkludert retning) bestemt av $a$. Prøv koden under:

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    return np.sin(x)
    # Legg inn funksjonen her

# Hastighet her
a=1    
    
x = np.arange(0, 2*np.pi, 0.01)

t = np.arange(0, 2*np.pi, 0.05)

# lager rutenettet
X, T = np.meshgrid(x,t)

# funksjonen på rutenettet
u = f(X-a*T)

In [ ]:
## animasjon

import matplotlib.animation as animation
from IPython.display import HTML

fig, ax = plt.subplots()

# initialplott, for tid t=0
line, = ax.plot(x, u[0,:])

# oppdater plott, for andre tider t_i
def animate(i):
    line.set_ydata(u[i,:])
    return line,

# animasjon
# interval bestemmer hvor fort det vises
# save_count hvor mange tider t_i som inngår
ani = animation.FuncAnimation(fig, animate, interval=20, blit=True, save_count=t.size)


HTML(ani.to_jshtml())

Det er nyttig å se på en kontourplott. Kontourene har et spesielt navn: **karakteristikker**. 

Siden vi vet verdien av løsningen fra de initialbetingelsene, vet vi verdien av hver kontour gjennom $x$-aksen. Da kan vi finne løsningen hvor som helst ved å spore langs karakteristikkene.

In [ ]:
# kontourplott

plt.contour(X,T,u)

### b) Med randbetingelser

Et typisk randverdiproblem er:

$$
u_t + a u_x = 0, \quad 0<x<1, \quad u(x,0)=f(x), \quad  u(0,t)=h(t)
$$

Løsningen er fortsett $u(x,t) = g(x-at)$, men ved å sette $x=0$ får vi $u(0,t) = g(-at)$, slik at det ser ut som vi burde heller tatt $g(-at) = h(t)$, og dermed

**Alternativ 1**
$$
u(x,t) = h\left( -\frac{x-at}{a}\right)
$$

Men hva med den gamle løsningen

**Alternativ 2**
$$
u(x,t) = f(x-at)?
$$

Vi kan forstå svaret bedre med å se på kontourene. Det viser seg at $u(x,t)$ er et av de to alternativene over, avhengig av verdien til $x, t$. 


**Oppgave**:
Prøv å forstå fra plotten under at alternativ 1 inntreffer om $x < at$, mens om $x > at$ får vi alternativ 2.

In [ ]:
## nytt rutenett

x = np.arange(0, 1, 0.01)

t = np.arange(0, 1, 0.01)

# lager rutenettet
X, T = np.meshgrid(x,t)

## kontourplott

def h(x):
    return 2+np.sin(x)

def u(x,t):
    return np.where(x > a*t, f(x-a*t), h(-(x-a*t)/a))

v = u(X,T)

# Hvilken verdier vi ønsker å se på nivåkurvene til
levels = np.linspace(-1,3,20)

plt.contour(X,T,v, levels=levels)

## 2. En numerisk løsning

Vår 'fasit' blir metoden:

$$
u^{n+1}_m = \frac{1}{2}(u^n_{m+1} + u^n_{m-1}) + \frac{ak}{2h} u^n_{m-1} - \frac{ak}{2h} u^n_{m+1}
$$

som heter **Lax-Friedrichs metode**. Det er ikke nødvendigvis den beste metoden, men den er pålitelig, lite krevende og implementere, og kan lett adapteres til mer generelle bevaringslover (se senere).

In [16]:
## Lax-Friedrichs

# a hastighet
# fa randbetingelser på venstre side, en funksjon av tid
# fb randbetingelser på høyre side, en funksjon av tid
# h steglengde i rom
# k steglengde i tid
# t nåværende tid
# x0 nåværende status, altså u(x_m)
# return x1, status på neste tidspunkt, altså u(x_m+1)

def lf(a, fa, fb, x0, h, k, t):
    n = x0.size
    x1 = np.zeros(n)
    for i in np.arange(1,n-1):
        x1[i] = (x0[i+1]+x0[i-1])/2 + (a*k/(2*h))*x0[i-1] - (a*k/(2*h))*x0[i+1]
    x1[0] = fa(t)
    x1[-1] = fb(t)
    return x1

In [23]:
# randbetingelser venstre side

def fa(t):
    return np.sin(np.pi*t)

# randbetingelser høyre side

def fb(t):
    return np.sin(np.pi*t)

# setter hastighet a

hastighet = 1

# setter opp gitteret med punkter i tid og rom
# 200 punkter mellom 0 og 1 i tid
# 50 punkter mellom 1 og -1 i rom

T = 1
n = 200

a = -1
b = 1
m = 50

t = np.linspace(0,T,n)
x = np.linspace(a,b,m)

# beregn k og h

k = t[1]-t[0]
h = x[1]-x[0]

# initialiserer u
u = np.zeros((n,m))

# initialbetingelser
u[0,:] = np.sin(np.pi*x)


# fjern kommentarer fra metoden du vil bruke

for i in np.arange(1,n):
    u[i,:] = lf(hastighet, fa, fb, u[i-1,:],h,k,t[i])    

In [ ]:
fig, ax = plt.subplots()

line, = ax.plot(x, u[0,:])

def animate(i):
    line.set_ydata(u[i,:])
    return line,

ani = animation.FuncAnimation(fig, animate, interval=20, blit=True, save_count=t.size)

from IPython.display import HTML
HTML(ani.to_jshtml())